In [ ]:
import pandas as pd
import os
from datetime import datetime
import re
import utils
import numpy as np
from main_openfists_feet import split_data_by_subject, epoch_all_subject_events

In [68]:
data_folder = "/Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2"
def extract_dt(path):
    # sub-EB-43_EEG_recording_2025-06-08-19.57.52.csv
    ts = re.search(r"(\d{4}-\d{2}-\d{2}-\d{2}\.\d{2}\.\d{2})", path).group(1)
    return datetime.strptime(ts, "%Y-%m-%d-%H.%M.%S")


eeg_paths = []
for fname in os.listdir(data_folder):
    if "EEG" in fname and ".csv" in fname:
        eeg_paths.append(os.path.join(data_folder, fname))
eeg_paths.sort(key=extract_dt)
exp1, exp2 = [], []
for idx, fp in enumerate(eeg_paths): #0,1,2,3
    (exp1 if idx % 2 == 0 else exp2).append([fp]) #even number is for open/close fists

print("Experiment-1 (fists) files:", exp1) #open/close fists
print("Experiment-2 (feet)  files:", exp2) #open/close feet



Experiment-1 (fists) files: [['/Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-19.46.10.csv'], ['/Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-20.11.07.csv']]
Experiment-2 (feet)  files: [['/Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-19.57.52.csv'], ['/Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-20.24.28.csv']]


In [69]:
CODEMAP = {
    1: {"target": 101, "non-target": 201, "prefix": "openclosefists"},
    2: {"target": 102, "non-target": 202, "prefix": "openclosefeet"},
}

In [70]:
filter=False
h_freq=30.0
l_freq=0.0

In [71]:
exp_1 = split_data_by_subject(exp1)
exp_2 = split_data_by_subject(exp2)
epochs_list_1 = epoch_all_subject_events(
    exp_1, CODEMAP[1], 1, tmax=4, filter=filter, h_freq=h_freq, l_freq=l_freq
)
epochs_list_2 = epoch_all_subject_events(
    exp_2, CODEMAP[2], 2, tmax=4, filter=filter, h_freq=h_freq, l_freq=l_freq
)


Loaded /Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-19.46.10.csv with shape (69192, 6)
Creating RawArray with float64 data, n_channels=5, n_times=69192
    Range : 0 ... 69191 =      0.000 ...   270.277 secs
Ready.
Loaded /Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-20.11.07.csv with shape (117840, 6)
Creating RawArray with float64 data, n_channels=5, n_times=117840
    Range : 0 ... 117839 =      0.000 ...   460.309 secs
Ready.
Loaded /Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-19.57.52.csv with shape (114744, 6)
Creating RawArray with float64 data, n_channels=5, n_times=114744
    Range : 0 ... 114743 =      0.000 ...   448.215 secs
Ready.
Loaded /Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/Muse_data_OpenCloseFeet 2/sub-EB-43_EEG_recording_2025-06-08-2

In [ ]:
from itertools import chain

OUT_ROOT = "/Users/hiro/Desktop/fun-website/decoder/EB-Data/sub-EB-43/" \
           "Muse_data_OpenCloseFeet 2/Muse_data_OpenCloseFeet_segmented"

os.makedirs(OUT_ROOT, exist_ok=True)         

PREFIX_MAP  = {1: "openclosefists", 2: "openclosefeet"}
CHANNELS    = ["TP9", "AF7", "AF8", "TP10"]
TARGET_CODE = "Target"

CODEMAP = {                            
    1: {"target": 101, "non-target": 201},
    2: {"target": 102, "non-target": 202},
}

def export_epochs_to_csv(
        epochs_list,
        prefix: str,
        out_dir: str,
        channels=CHANNELS,
        target_label: str = TARGET_CODE,
):

    os.makedirs(out_dir, exist_ok=True)

    # flatten to 1D
    flat_epochs = (list(chain.from_iterable(epochs_list))
                   if isinstance(epochs_list[0], (list, tuple))
                   else epochs_list)

    for run_idx, epochs in enumerate(flat_epochs, start=1):     # run1, run2…
        # create channel csv files
        for ch in channels:
            data = epochs.get_data(picks=[ch])        # (n_events, 1, 1024)
            arr  = np.squeeze(data, axis=1)           # (n_events, 1024)
            out_path = os.path.join(out_dir,
                          f"{prefix}_run{run_idx}_{ch}.csv")
            pd.DataFrame(arr).to_csv(out_path, index=False, header=False)

        # label csv
        labels = (epochs.events[:, 2] == epochs.event_id[target_label]).astype(int)
        lab_path = os.path.join(out_dir, f"run_{run_idx}_label.csv")
        pd.Series(labels).to_csv(lab_path, index=False, header=False)

        print(f"[{prefix}] run{run_idx}: exports {len(labels)}  × 1024 samples")

export_epochs_to_csv(
    epochs_list_1,
    prefix=PREFIX_MAP[1],
    out_dir=os.path.join(OUT_ROOT, "exp_1")   # fists
)

export_epochs_to_csv(
    epochs_list_2,
    prefix=PREFIX_MAP[2],
    out_dir=os.path.join(OUT_ROOT, "exp_2")   # feet
)


[openclosefists] run1: 导出 63 段 × 1024 samples
[openclosefists] run2: 导出 100 段 × 1024 samples
[openclosefeet] run1: 导出 100 段 × 1024 samples
[openclosefeet] run2: 导出 100 段 × 1024 samples
